In [ ]:
!pip install -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

In [2]:
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer
)
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
huggingface_dataset_name = "yasminesarraj/texts_summary"

dataset = load_dataset(huggingface_dataset_name)
print (dataset)

Generating train split:   0%|          | 0/47 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/47 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'summary'],
        num_rows: 47
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'summary'],
        num_rows: 47
    })
})


In [4]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
    use_fast=False
    )
eval_tokenizer.pad_token = eval_tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [7]:

def gen(model,p, maxlen=1000, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"),
                         max_new_tokens=maxlen,
                         pad_token_id=eval_tokenizer.eos_token_id,
                         do_sample=sample,
                         num_return_sequences=1,
                         temperature=0.1,
                         num_beams=1,
                         top_p=0.95,
                         ).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(
    base_model,
    "/content/drive/MyDrive/checkpoint-400",
    torch_dtype=torch.float16,
    is_trainable=False
    )

In [12]:
%%time
import pandas as pd
from transformers import set_seed
import textwrap

seed = 42

set_seed(seed)



txt_list = dataset['test'][21:23]['text']
summaries = dataset['test'][21:23]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []


for idx, text in enumerate(txt_list):
  txt = txt_list[idx]
  summary = summaries[idx]

  prompt = f"Instruct: Summarize the following scientific reports .\n{txt}\nOutput:\n"


  peft_model_res = gen(ft_model, prompt, 100,)
  peft_model_output = peft_model_res[0].split('Output:\n')[1]
  peft_model_text_output, success, result = peft_model_output.partition('#End')


  original_model_res = gen(base_model, prompt, 100,)
  original_model_text_output = original_model_res[0].split('Output:\n')[1]

  original_model_summaries.append(original_model_text_output)
  peft_model_summaries.append(peft_model_text_output)
  instruct_model_summaries.append(summary)


zipped_summaries = list(zip(instruct_model_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])

df



Token indices sequence length is longer than the specified maximum sequence length for this model (2156 > 2048). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


CPU times: user 45.1 s, sys: 1.49 s, total: 46.6 s
Wall time: 51.7 s


,baseline_summaries,original_model_summaries,peft_model_summaries
0,"This paper proposes Swim, a new activation fu...","This paper presents Swim, a new activation fu...","This paper presents Swim, a new activation fu..."
1,This paper provides an overview of Ensemble R...,A simple agent-based model of a mathematical f...,A simple agent-based model of a mathematical f...


In [14]:
import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=original_model_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=original_model_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

ORIGINAL MODEL:
{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
PEFT MODEL:
{'rouge1': 0.6822916666666667, 'rouge2': 0.5128626163108921, 'rougeL': 0.5786458333333334, 'rougeLsum': 0.6822916666666667}
Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: -31.77%
rouge2: -48.71%
rougeL: -42.14%
rougeLsum: -31.77%


In [ ]:
#model = AutoModelForCausalLM.from_pretrained(base_model_name)
#model = PeftModel.from_pretrained(model, adapter_model_name)

model = ft_model.merge_and_unload()
model.save_pretrained("merged_adapters")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
